In [3]:
import torch
import torch.nn as nn
# import torchvision.transforms as transforms

In [ ]:
from matplotlib.style import context
import torch
import torch.nn as nn
import torch.nn.functional as F

import create_vocab
import data_to_tensors

import random 

from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence, pack_sequence


import pandas as pd
from IPython.display import display

import matplotlib.pyplot as plt
import pandas as pd

import torch.optim as optim

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# import seaborn as sns
########################################################################

data_folder = 'Codes_embedding'
dataset_name = 'Codes_embedding'

dict_path = 'data/'+ data_folder + '/' + dataset_name + '.dict.c2v'
word2idx, path2idx, target2idx, idx2target = create_vocab.create_vocab(dict_path)

path_for_train = 'data/'+ data_folder + '/' + dataset_name + '.train.c2v'
train_dataset = data_to_tensors.TextDataset(path_for_train, 
                                                    word2idx, 
                                                    path2idx, 
                                                    target2idx)

path_for_val = 'data/'+ data_folder + '/' + dataset_name + '.val.c2v'
val_dataset = data_to_tensors.TextDataset(path_for_val, 
                                                    word2idx, 
                                                    path2idx, 
                                                    target2idx)

path_for_test = 'data/'+ data_folder + '/' + dataset_name + '.test.c2v'
test_dataset = data_to_tensors.TextDataset(path_for_test, 
                                                    word2idx, 
                                                    path2idx, 
                                                    target2idx)

train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=512, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)


################################################################

class LSTM(nn.Module):

    def __init__(self, 
               val_embedding_dim = 128,
               path_embedding_dim = 128,
               dropout_rate = 0.25,
               embedding_dim = 128,
               values_vocab_size = 0,
               paths_vocab_size = 0,
               labels_num = 0):
        super(LSTM, self).__init__()

        self.values_vocab_size = values_vocab_size
        self.paths_vocab_size = paths_vocab_size
        self.val_embedding_dim = val_embedding_dim
        self.path_embedding_dim = path_embedding_dim
        self.dropout_rate = dropout_rate
        self.embedding_dim = embedding_dim
        self.labels_num = labels_num

        self.values_embedding = nn.Embedding(self.values_vocab_size, self.val_embedding_dim)
        self.paths_embedding = nn.Embedding(self.paths_vocab_size, self.path_embedding_dim)
        self.dimension = embedding_dim
        
        self.output_linear = nn.Linear(self.embedding_dim, 1, bias = False)

        self.lstm = nn.LSTM(input_size=128,
                            hidden_size=128,
                            num_layers=1,
                            batch_first=True,
                            bidirectional=True)
        self.drop = nn.Dropout(p=0.5)

        self.fc = nn.Linear(2*embedding_dim, 1)

        # self.linear = nn.Linear(self.path_embedding_dim + 2 * self.val_embedding_dim, self.embedding_dim, bias = False)
        # self.linear = nn.Linear(128 + 2 * 128, 128, bias = False)
        self.linear = nn.Linear(128, 128, bias = False)
        #self.linear = nn.Linear(self.path_embedding_dim + 2 * self.val_embedding_dim, self.embedding_dim, bias = False)

    # def forward(self, text, text_len):
    def forward(self, starts, paths, ends):

        # text_emb = self.embedding(text)
        start_embedding = self.values_embedding(starts)
        # [1,200,128]
        path_embedding = self.paths_embedding(paths)
        end_embedding = self.values_embedding(ends)
        # [1,200,128]

        context_vec = torch.cat((start_embedding, path_embedding, end_embedding))
        print("CONTEXT VEC",context_vec.shape)

        comb_context_vec = torch.tanh(self.linear(context_vec))
        print(type(comb_context_vec))
        print("COMB CONTEXT VEC",comb_context_vec.shape)
        # [3, 200, 128]
        # [batch, seq, features]

        # packed_input = pack_padded_sequence(context_vec, torch.as_tensor(len(word2idx), dtype=torch.int64), batch_first=True, enforce_sorted=False)
        #packed_input = pack_sequence(context_vec) #'torch.nn.utils.rnn.PackedSequence'
        # print(type(packed_input))
        torch.set_printoptions(profile="full")
        # print(packed_input.data)
        packed_output, _ = self.lstm(comb_context_vec)
        # output, _ = pad_sequence(packed_output)
        output = packed_output
        # output = self.output_linear(packed_output)
        print("outputttt ",output.size())
        # print(len(output))
        # exit()

        # out_forward =  output[range(len(output)), 128 - 1, :self.dimension]
        # # exit()
        # # out_forward = output[range( len(word2idx)) ,self.dimension]
        # out_reverse = output[:, 0, self.dimension:]
        # out_reduced = torch.cat((out_forward, out_reverse), 1)
        # path_fea = self.drop(out_reduced)

        # path_fea = self.fc(path_fea)
        # path_fea = torch.squeeze(path_fea, 1)
        # label_out = torch.sigmoid(path_fea)

        # label_out = torch.sigmoid((output.data))
        print('Label out',len(output))
        return output

# Save and Load Functions

def save_checkpoint(save_path, model, optimizer, valid_loss):

    if save_path == None:
        return
    
    state_dict = {'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'valid_loss': valid_loss}
    
    torch.save(state_dict, save_path)
    print(f'Model saved to ==> {save_path}')


def load_checkpoint(load_path, model, optimizer):

    if load_path==None:
        return
    
    state_dict = torch.load(load_path, map_location="cpu")
    print(f'Model loaded from <== {load_path}')
    
    model.load_state_dict(state_dict['model_state_dict'])
    optimizer.load_state_dict(state_dict['optimizer_state_dict'])
    
    return state_dict['valid_loss']


def save_metrics(save_path, train_loss_list, valid_loss_list, global_steps_list):

    if save_path == None:
        return
    
    state_dict = {'train_loss_list': train_loss_list,
                'valid_loss_list': valid_loss_list,
                'global_steps_list': global_steps_list}
    
    torch.save(state_dict, save_path)
    print(f'Model saved to ==> {save_path}')


def load_metrics(load_path):

    if load_path==None:
        return
    
    state_dict = torch.load(load_path, map_location="cpu")
    print(f'Model loaded from <== {load_path}')
    
    return state_dict['train_loss_list'], state_dict['valid_loss_list'], state_dict['global_steps_list']

# Training Function

def train(model,
          optimizer,
        #   criterion = nn.BCELoss(),
          criterion = nn.CrossEntropyLoss(),
          train_loader = train_loader,
          valid_loader = val_loader,
          num_epochs = 5,
          eval_every = len(train_loader) // 2,
          file_path = "lstm_model",
          best_valid_loss = float("Inf")):
    
    # initialize running values
    running_loss = 0.0
    valid_running_loss = 0.0
    global_step = 0
    train_loss_list = []
    valid_loss_list = []
    global_steps_list = []

    # training loop
    model.train()
    for epoch in range(num_epochs):
        # for (labels, (title, title_len), (text, text_len), (titletext, titletext_len)), _ in train_loader:           
        for starts, contexts, ends, labels in train_loader:
            print("Train ka for labels",labels)
            # labels = labels.to("cpu")
            starts, contexts, ends = starts.to("cpu"), contexts.to("cpu"), ends.to("cpu")
            # output = model(titletext, titletext_len)
            labels = labels.to("cpu")
            print(type(starts))
            print(type(ends))
            print(type(contexts))
            # _, y_pred = model(starts, contexts, ends)
            y_pred = model(starts, contexts, ends)
            # y_pred.shape = [600, 1024]

            print(y_pred.shape, '****************************************************************')
            print('-------', labels.size())

            loss = criterion(y_pred, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # update running values
            running_loss += loss.item()
            global_step += 1

            # evaluation step
            if global_step % eval_every == 0:
                model.eval()
                with torch.no_grad():                    
                  # validation loop
                  for (labels, (title, title_len), (text, text_len), (titletext, titletext_len)), _ in valid_loader:
                      labels = labels.to("cpu")
                      titletext = titletext.to("cpu")
                      titletext_len = titletext_len.to("cpu")
                      output = model(titletext, titletext_len)

                      loss = criterion(output, labels)
                      valid_running_loss += loss.item()

                # evaluation
                average_train_loss = running_loss / eval_every
                average_valid_loss = valid_running_loss / len(valid_loader)
                train_loss_list.append(average_train_loss)
                valid_loss_list.append(average_valid_loss)
                global_steps_list.append(global_step)

                # resetting running values
                running_loss = 0.0                
                valid_running_loss = 0.0
                model.train()

                # print progress
                print('Epoch [{}/{}], Step [{}/{}], Train Loss: {:.4f}, Valid Loss: {:.4f}'
                      .format(epoch+1, num_epochs, global_step, num_epochs*len(train_loader),
                              average_train_loss, average_valid_loss))
                
                # checkpoint
                if best_valid_loss > average_valid_loss:
                    best_valid_loss = average_valid_loss
                    save_checkpoint(file_path + '/model.pt', model, optimizer, best_valid_loss)
                    save_metrics(file_path + '/metrics.pt', train_loss_list, valid_loss_list, global_steps_list)
    
    save_metrics(file_path + '/metrics.pt', train_loss_list, valid_loss_list, global_steps_list)
    print('Finished Training!')


model = LSTM().to("cpu")
optimizer = optim.Adam(model.parameters(), lr=0.001)

train(model=model, optimizer=optimizer, num_epochs=10)

In [21]:
import create_vocab
import data_to_tensors
import model_implementation
from train_class import TrainingModule

import torch
import torch.nn as nn
import torch.nn.functional as F
import random 

from torch.utils.data import DataLoader

import pandas as pd
from IPython.display import display

from sklearn.metrics import accuracy_score

def main():

    N_EPOCHS = 40
    LR = 3e-3
    WD = 0.8e-5

    SEED = 1337
    random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.backends.cudnn.deterministic = True
    
    data_folder = 'Codes_embedding'
    dataset_name = 'Codes_embedding'

    # data_folder = 'CODES_emb'
    # dataset_name = 'CODES_emb'

    dict_path = 'data/'+ data_folder + '/' + dataset_name + '.dict.c2v'
    word2idx, path2idx, target2idx, idx2target = create_vocab.create_vocab(dict_path)

    path_for_train = 'data/'+ data_folder + '/' + dataset_name + '.train.c2v'
    train_dataset = data_to_tensors.TextDataset(path_for_train, 
                                                        word2idx, 
                                                        path2idx, 
                                                        target2idx)

    path_for_val = 'data/'+ data_folder + '/' + dataset_name + '.val.c2v'
    val_dataset = data_to_tensors.TextDataset(path_for_val, 
                                                        word2idx, 
                                                        path2idx, 
                                                        target2idx)

    path_for_test = 'data/'+ data_folder + '/' + dataset_name + '.test.c2v'
    test_dataset = data_to_tensors.TextDataset(path_for_test, 
                                                        word2idx, 
                                                        path2idx, 
                                                        target2idx)

    train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=512, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

#     print(len(word2idx))
#     print(word2idx)
    print(len(path2idx))
    print(path2idx)
#     print(len(target2idx))
#     print(len(idx2target))
#     print(target2idx)
#     print(idx2target)


#     model = model_implementation.code2vec_model(values_vocab_size = len(word2idx), 
#                             paths_vocab_size = len(path2idx), 
#                             labels_num = len(target2idx))
    

#     optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WD)
#     criterion = nn.CrossEntropyLoss()

#     train_class = TrainingModule(model, optimizer, criterion, train_loader, val_loader, test_loader, N_EPOCHS, idx2target)
#     list_train_loss, list_val_loss, list_train_precision, list_val_precision,list_train_recall, list_val_recall, list_train_f1, list_val_f1,list_train_accuracy,list_val_accuracy = train_class.train(dataset_name)

    
#     state_dict = torch.load( './' + dataset_name + '_article_model.pth')

#     model.load_state_dict(state_dict)

#     DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#     model = model.to(DEVICE)

#     d = {'Original names': [], 'Predicted names': []}

#     for start, path, end, label in iter(test_loader):
#         # get from model
#         code, y_pred = model(start.to(DEVICE), path.to(DEVICE), end.to(DEVICE))
#         # get probability
#         y_pred = F.softmax(y_pred)
#         # get best name index
#         y_pred = torch.argmax(y_pred, dim = 1)
        
#         for i, j in zip(label, y_pred):
            
#             d['Original names'].append(idx2target[i.item()])
#             d['Predicted names'].append(idx2target[j.item()])
#             # break

#     df = pd.DataFrame(data=d)
#     display(df,)

#     print("Accuracy : ",accuracy_score(df['Original names'], df['Predicted names']))

if __name__== "__main__":
    
    # batch_size = int(input('Input batch size: '))
    batch_size = 1024
    # lr = float(input('Input learning rate: '))
    lr = 0.01
    # wd = float(input('Input weight decay: '))
    wd = 0.01

    main()

10499
{'<unk>': 0, '<pad>': 1, '0006b0bd0f5f22a2ba7c95349c7c4f6009ad2b8fa9788715c2d7a745f4a8b85d': 2, '000c43227fcf9560fcd4d1a518d2d12b41261422e55afc2915fa7f98f7ecd4da': 3, '00142beab7d30fe0514f1081adfdf7b0dbec9f90882c636672455e1953c66b88': 4, '001fc0b57f2b6047c5af893586f9a89a186f0e7dae5633aba5713e8a344605e1': 5, '002978fa96464f63e8f84343ff2afbe7a59e6de543995fac34bf6673908e1f6e': 6, '002a299bec3a3b80fc3b32b9e2ef9342a6fe55bc381aa43c9442f80fdbcedcc4': 7, '002c51c4d8125c60eacd1ee0c98b3cf254f8038498645c211ac9e5338a30281e': 8, '002cac81ea00f5cb4b333781f96e65aae8daa9a274f526e3a9c23863cd8994d9': 9, '002d6301760ee32abe7a8ee01660be616c24e1dd3f1ec6321cecfa0fa3428289': 10, '0031afe7fced8f8612bf26fb3ff341b4b2d04f8310c7e5c38226a5b7129e004f': 11, '00353290fc542f4992f0b31d27021c815c9d6799dd74fcc8f427236e94e9588e': 12, '003699540175b9f8e3b87587d8873fc47905353288658f3c715da4d812b51880': 13, '00387d0729776ad5e39c392b20f0d9df2bd98e55e4c63c1a173283b910ef2334': 14, '003ba7bbb7b2de9f52ffcf0702230f4caa56185a